<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/083_Agent_System_Components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# ✅ Agent System Components (with Planning & Reflection)

1. **Tool** – Function that does something useful (e.g., `create_plan`, `track_progress`, domain tools)
2. **Tool Registry** – Stores all tools by name for lookup by the agent
3. **ActionContext** – Holds dependencies (e.g., memory, auth, services, registry)
4. **Environment** – Executes tools using the ActionContext for dependency injection
5. **Agent** – Parses input, chooses tools, calls them via Environment
6. **Capabilities** – Modular behaviors that hook into the agent loop (e.g., **PlanFirstCapability**, **ProgressTrackingCapability**)
7. **Dependencies** – Injected values (e.g., SMTP, tokens, LLM) automatically provided to tools
8. **Wiring** – Instantiate and connect everything together (Agent + Environment + Capabilities + Tools)




# 🧩 Agent-Building Checklist (with Planning & Reflection)

Use this as your repeatable recipe from blank file → running agent.

## 1) Define intent

* **Goal statement:** what success looks like.
* **Constraints:** time, budget, safety, privacy.
* **Inputs/outputs:** formats, APIs, files.

## 2) Map capabilities

* **Domain tools:** what actions the agent must perform (email, DB, calendar, etc.).
* **Lifecycle capabilities:**

  * **PlanFirstCapability** (create plan before acting)
  * **ProgressTrackingCapability** (reflect at end of loop)

## 3) Design tools

* Implement tools with `@register_tool`.
* Follow DI conventions:

  * First arg `action_context` (if needed).
  * Injected deps use **underscore** params (e.g., `_auth_token`).
* Keep tools **stateless** and narrow.

## 4) Build the Tool Registry

* Register all tools under clear, unique names.
* Include planning/reflecting tools: `create_plan`, `track_progress`.

## 5) Assemble the ActionContext

* Put shared deps here (single source of truth):

  * `memory`, `llm`, `auth_token`, `user_config`, service clients, registry.
* Decide what’s **session/request-specific** vs **global defaults**.

## 6) Configure the Environment

* Environment handles **dependency injection**:

  * Auto-inject `action_context` if requested.
  * Match `_dep_name` params from context keys.
* Keep the agent free of DI logic.

## 7) Compose the Agent

* Provide: goals, language, registry, environment, `generate_response`.
* Attach capabilities:

  * `PlanFirstCapability(plan_memory_type="system", track_progress=True)`
  * `ProgressTrackingCapability(track_frequency=1 or 2)`
* Set sensible limits (`max_iterations`, timeouts).

## 8) Wire & run

* Instantiate dependencies → `ActionContext` → `Environment` → `Agent`.
* Optionally pass `action_context_props` to **override/extend** per request.
* Call `agent.run(user_input, action_context_props=...)`.

## 9) Test the loop

* Unit test tools (pure functions).
* Dry-run the agent:

  * Verify plan appears in memory first.
  * Watch progress reports append each iteration.
* Add guardrails: input validation, safe defaults, error handling.

## 10) Iterate & harden

* Tune prompts for **plan** and **progress** tools.
* Adjust `track_frequency` for cost/latency.
* Add logging/metrics capabilities (optional).
* Version tools & capabilities; keep changelogs.

---

### Quick “Starter Pack” (minimum set)

* Tools: your domain tools + `create_plan` + `track_progress`
* Capabilities: `PlanFirstCapability`, `ProgressTrackingCapability`
* Context keys: `memory`, `llm`, (plus any `_deps` your tools require)
* Environment: DI enabled (underscore injection + action\_context support)



#Agent Scaffold

* tools + registry
* action context (dependency injection)
* environment (auto-injects deps)
* memory
* capabilities: **PlanFirst** + **ProgressTracking**
* a minimal agent loop

> it uses a tiny fake LLM so it runs without external services.


In [ ]:

# =========================
# 0) Minimal Infrastructure
# =========================
from dataclasses import dataclass
from typing import Callable, Dict, List, Any
import inspect
from functools import reduce
import uuid

# --- Memory ---
class Memory:
    def __init__(self):
        self.items: List[Dict[str, Any]] = []

    def add_memory(self, item: Dict[str, Any]):
        self.items.append(item)

    def get_memories(self):
        return self.items

# --- ActionContext (Dependency Holder) ---
class ActionContext:
    def __init__(self, properties: Dict[str, Any] = None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self) -> Memory:
        return self.get("memory")

    def get_action_registry(self):
        return self.get("action_registry")

# --- Tool registration ---
@dataclass
class ActionDef:
    name: str
    description: str
    function: Callable

class ActionRegistry:
    def __init__(self):
        self._actions: Dict[str, ActionDef] = {}

    def register(self, name: str, description: str, fn: Callable):
        self._actions[name] = ActionDef(name=name, description=description, function=fn)

    def get(self, name: str) -> ActionDef:
        return self._actions[name]

    def get_actions(self) -> List[ActionDef]:
        return list(self._actions.values())

# tiny decorator
def register_tool(description: str):
    def deco(fn: Callable):
        fn.__tool_description__ = description
        return fn
    return deco

# --- Environment (executes tools + DI) ---
class Environment:
    def execute_action(self, action_def: ActionDef, action_context: ActionContext, args: Dict[str, Any]):
        args_copy = dict(args)

        # inject action_context if the tool asks for it
        if "action_context" in inspect.signature(action_def.function).parameters:
            args_copy["action_context"] = action_context

        # underscore DI: _key <- context["key"]
        for key, value in action_context.properties.items():
            injected_param = f"_{key}"
            if injected_param in inspect.signature(action_def.function).parameters:
                args_copy[injected_param] = value

        return action_def.function(**args_copy)

# --- Capability base ---
class Capability:
    def init(self, agent, action_context: ActionContext):
        pass
    def start_agent_loop(self, agent, action_context: ActionContext):
        return True
    def process_prompt(self, agent, action_context: ActionContext, prompt: str) -> str:
        return prompt
    def end_agent_loop(self, agent, action_context: ActionContext):
        pass
    def should_terminate(self, agent, action_context: ActionContext, response: str) -> bool:
        return False

# --- A tiny "LLM" you can swap out later ---
def fake_llm(prompt: str) -> str:
    # Obviously simplistic; replace with a real LLM call later.
    if "Create a detailed plan" in prompt or "create a detailed plan" in prompt:
        return "1) Greet user\n2) Do task X\n3) Summarize result"
    if "progress report" in prompt:
        return "Progress: greeted user. Next: Do task X. No blockers."
    return f"(LLM response to) {prompt[:80]}..."

# Helper to expose LLM to tools
def prompt_llm(action_context: ActionContext, prompt: str) -> str:
    llm = action_context.get("llm", fake_llm)
    return llm(prompt)

# ================
# 1) Domain Tools
# ================
@register_tool(description="Say hello to a person")
def say_hello(action_context: ActionContext, name: str, _greeting_prefix: str) -> str:
    # underscore DI injects _greeting_prefix from context['greeting_prefix']
    return f"{_greeting_prefix} {name}!"

# Planning tool
@register_tool(description="Create a plan from goals and available tools")
def create_plan(action_context: ActionContext, memory: Memory, action_registry: ActionRegistry) -> str:
    tool_list = "\n".join(f"- {a.name}: {a.description}" for a in action_registry.get_actions())
    context = "\n".join(f"{m.get('type')}: {m.get('content')}" for m in memory.get_memories() if m.get("type") in ("user","system"))
    prompt = f"""Given the task in memory and the available tools, create a detailed plan.
Think step by step. List numbered steps with tool choices and expected outcomes.

Available tools:
{tool_list}

Task context:
{context}
"""
    return prompt_llm(action_context, prompt)

# Reflection tool
@register_tool(description="Create a progress report from the latest state")
def track_progress(action_context: ActionContext, _memory: Memory, action_registry: ActionRegistry) -> str:
    tool_list = ", ".join(a.name for a in action_registry.get_actions())
    context = "\n".join(f"{m.get('type')}: {m.get('content')}" for m in _memory.get_memories() if m.get("type") in ("user","system"))
    prompt = f"""Given current task and tools ({tool_list}), generate a concise progress report:
- What was done
- Blockers
- Next steps

Task context:
{context}
"""
    return prompt_llm(action_context, prompt)

# ==================
# 2) Capabilities
# ==================
class PlanFirstCapability(Capability):
    def __init__(self, plan_memory_type="system"):
        self.plan_memory_type = plan_memory_type
        self.did_init = False

    def init(self, agent, action_context: ActionContext):
        if self.did_init:
            return
        self.did_init = True

        plan = create_plan(
            action_context=action_context,
            memory=action_context.get_memory(),
            action_registry=action_context.get_action_registry()
        )
        action_context.get_memory().add_memory({
            "type": self.plan_memory_type,
            "content": "Plan:\n" + plan
        })

class ProgressTrackingCapability(Capability):
    def __init__(self, memory_type="system", track_frequency=1):
        self.memory_type = memory_type
        self.track_frequency = track_frequency
        self.iteration = 0

    def end_agent_loop(self, agent, action_context: ActionContext):
        self.iteration += 1
        if self.iteration % self.track_frequency != 0:
            return
        mem = action_context.get_memory()
        report = track_progress(action_context=action_context, _memory=mem, action_registry=action_context.get_action_registry())
        mem.add_memory({"type": self.memory_type, "content": f"Progress (iter {self.iteration}):\n{report}"})


# ============
# 3) Agent
# ============
class Agent:
    def __init__(self, environment: Environment, registry: ActionRegistry, capabilities: List[Capability] = None, max_iterations: int = 3):
        self.environment = environment
        self.registry = registry
        self.capabilities = capabilities or []
        self.max_iterations = max_iterations

    def run(self, user_input: str, action_context_props: Dict[str, Any] = None):
        # Build ActionContext
        base_context = self.environment.action_context.properties
        merged = {**base_context, **(action_context_props or {})}
        action_context = ActionContext(merged)
        action_context.properties["action_registry"] = self.registry  # make registry available to tools

        # seed memory with the user request
        mem = action_context.get_memory()
        mem.add_memory({"type": "user", "content": user_input})

        # init capabilities
        for c in self.capabilities:
            c.init(self, action_context)

        # tiny demo loop: greet then pretend to do more work
        for i in range(self.max_iterations):
            # let capabilities modify the “prompt” if they want (string here for simplicity)
            prompt = f"iteration {i+1}: do work"
            prompt = reduce(lambda p, c: c.process_prompt(self, action_context, p), self.capabilities, prompt)

            # choose an action (hardcoded for scaffold)
            if i == 0:
                act = self.registry.get("say_hello")
                result = self.environment.execute_action(act, action_context, {"name": "Team"})
                mem.add_memory({"type": "assistant", "content": f"Ran say_hello -> {result}"})
            else:
                mem.add_memory({"type": "assistant", "content": "Did some domain-specific work..."})

            # end-of-loop reflection
            for c in self.capabilities:
                c.end_agent_loop(self, action_context)

        return mem

# ==========================
# 4) Wire it all together
# ==========================
# registry
registry = ActionRegistry()
registry.register("say_hello", getattr(say_hello, "__tool_description__", "Say hello"), say_hello)
registry.register("create_plan", getattr(create_plan, "__tool_description__", "Create plan"), create_plan)
registry.register("track_progress", getattr(track_progress, "__tool_description__", "Track progress"), track_progress)

# context (dependencies)
context = ActionContext({
    "memory": Memory(),
    "llm": fake_llm,
    "greeting_prefix": "Hello",
})

# environment (with DI) — attach context for access inside execute_action
class DIEnvironment(Environment):
    def __init__(self, action_context: ActionContext):
        self.action_context = action_context

env = DIEnvironment(context)

# agent with capabilities
agent = Agent(
    environment=env,
    registry=registry,
    capabilities=[
        PlanFirstCapability(),
        ProgressTrackingCapability(track_frequency=1),
    ],
    max_iterations=2
)

# run it
final_memory = agent.run("Please greet the team, then outline next steps.")
for m in final_memory.get_memories():
    print(f"[{m['type']}] {m['content']}")

